In [ ]:
# Importing
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [19]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [26]:
# Initialize chat model
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [27]:
# Define tools
search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in Singapore?")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in Singapore?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Singapore', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Singapore', 'region': '', 'country': 'Singapore', 'lat': 1.293, 'lon': 103.856, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1751855412, 'localtime': '2025-07-07 10:30'}, 'current': {'last_updated_epoch': 1751854500, 'last_updated': '2025-07-07 10:15', 'temp_c': 30.4, 'temp_f': 86.7, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.7, 'wind_kph': 17.3, 'wind_degree': 145, 'wind_dir': 'SE', 'pressure_mb': 1011.0, 'pressure_in': 29.85, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 66, 'cloud': 75, 'feelslike_c': 36.2, 'feelslike_f': 97.2, 'windchill_c': 29.7, 'windchill_f': 85.4, 'heatindex_c': 34.4, 'heatindex_f': 94.0, 'dewpoint_c': 23.9, 'dewpoint_f': 75.0, 'vis_km': 10.0

In [28]:
# Enable mode to do tool calling
model_with_tools = model.bind_tools(tools)

In [29]:
# Test with normal message
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hi, how can I help you today?

Tool calls: []


In [30]:
# Test
query = "Search for the weather in Singapore"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in Singapore'}, 'id': '43fa1ee4-bf8a-4519-acb2-97322a8df29f', 'type': 'tool_call'}]


## Create Agent

In [31]:
# Create agent
agent_executor = create_react_agent(model, tools)

In [33]:
# Run agent
input_message = {"role": "user", "content": "Search for the weather in Singapore"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Singapore
================================== Ai Message ==================================
Tool Calls:
  tavily_search (ea5a2ec6-89af-4b1e-91cb-627084743144)
 Call ID: ea5a2ec6-89af-4b1e-91cb-627084743144
  Args:
    query: weather in Singapore
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Singapore", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Singapore", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Singapore', 'region': '', 'country': 'Singapore', 'lat': 1.293, 'lon': 103.856, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1751855505, 'localtime': '2025-07-07 10:31'}, 'current': {'last_updated_epoch': 1751855400, 'last_updated': '2025-07-07 10:30', 'temp_c': 30.2, 'temp_f': 86.4, 'is_day': 1, 'condition': {'text': 'Partly clou

In [34]:
# Streaming messages
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in Singapore
================================== Ai Message ==================================
Tool Calls:
  tavily_search (02306818-96ff-45c1-b3d2-7ae9974fe280)
 Call ID: 02306818-96ff-45c1-b3d2-7ae9974fe280
  Args:
    query: weather in Singapore
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Singapore", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Singapore", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Singapore', 'region': '', 'country': 'Singapore', 'lat': 1.293, 'lon': 103.856, 'tz_id': 'Asia/Singapore', 'localtime_epoch': 1751855505, 'localtime': '2025-07-07 10:31'}, 'current': {'last_updated_epoch': 1751855400, 'last_updated': '2025-07-07 10:30', 'temp_c': 30.2, 'temp_f': 86.4, 'is_day': 1, 'condition': {'text': 'Partly clou

In [35]:
# Streaming tokens
for step, metadata in agent_executor.stream({"messages": [input_message]}, stream_mode="messages"):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in Singapore| on July 7, 2025 is partly cloudy with a temperature| of 30.2°C (86.4°F).| The wind is from the Southeast at 17.3 kph. The humidity is 62%.|

### Adding in memory

In [36]:
memory = MemorySaver()

In [37]:
# Create agent with memory
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [38]:
input_message = {"role": "user", "content": "Hi, I'm Fay!"}
for step in agent_executor.stream({"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Fay!
================================== Ai Message ==================================

Hi Fay, nice to meet you! Is there anything I can help you with today?


In [39]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream({"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Fay.


Note: For new conversation, change the thread_id.